In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

In [12]:
import nltk

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [16]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [3]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

282 documents
69 classes ['alcohol_impact', 'bad_cholesterol', 'benefits_of_fiber', 'caffeine_effects', 'cholesterol_age', 'cholesterol_aging', 'cholesterol_alcohol', 'cholesterol_and_diet', 'cholesterol_check', 'cholesterol_checkup', 'cholesterol_children', 'cholesterol_complications', 'cholesterol_definition', 'cholesterol_diabetes', 'cholesterol_diet', 'cholesterol_exercise', 'cholesterol_fasting', 'cholesterol_fiber', 'cholesterol_foods', 'cholesterol_genetics', 'cholesterol_herbal', 'cholesterol_hereditary', 'cholesterol_levels', 'cholesterol_medical_help', 'cholesterol_medication', 'cholesterol_medications', 'cholesterol_monitoring', 'cholesterol_myth', 'cholesterol_myths', 'cholesterol_omega3', 'cholesterol_pregnancy', 'cholesterol_smoking', 'cholesterol_supplements', 'cholesterol_supplements_pregnancy', 'cholesterol_testing', 'cholesterol_treatment', 'cholesterol_weight', 'exercise_benefits', 'exercise_consistency', 'exercise_frequency', 'exercise_recommendation', 'fiber_import

In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1e-6,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(64, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [9]:
#fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)

Epoch 1/500
57/57 [==============================] - 1s 1ms/step - loss: 4.2370 - accuracy: 0.0035
Epoch 2/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2447 - accuracy: 0.0284
Epoch 3/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2384 - accuracy: 0.0106
Epoch 4/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2379 - accuracy: 0.0142
Epoch 5/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2457 - accuracy: 0.0071
Epoch 6/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2296 - accuracy: 0.0106
Epoch 7/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2389 - accuracy: 0.0284
Epoch 8/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2261 - accuracy: 0.0284
Epoch 9/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2337 - accuracy: 0.0284
Epoch 10/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2289 - accuracy: 0.0142
Epoch 11/

57/57 [==============================] - 0s 2ms/step - loss: 4.2245 - accuracy: 0.0106
Epoch 164/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2302 - accuracy: 0.0106
Epoch 165/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2291 - accuracy: 0.0142
Epoch 166/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2336 - accuracy: 0.0142
Epoch 167/500
57/57 [==============================] - 0s 2ms/step - loss: 4.2379 - accuracy: 0.0177
Epoch 168/500
57/57 [==============================] - 0s 2ms/step - loss: 4.2385 - accuracy: 0.0177
Epoch 169/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2423 - accuracy: 0.0248
Epoch 170/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2271 - accuracy: 0.0284
Epoch 171/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2445 - accuracy: 0.0106
Epoch 172/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2164 - accuracy: 0.0319
Epoc

57/57 [==============================] - 0s 1ms/step - loss: 4.2382 - accuracy: 0.0142
Epoch 326/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2413 - accuracy: 0.0248
Epoch 327/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2408 - accuracy: 0.0142
Epoch 328/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2374 - accuracy: 0.0177
Epoch 329/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2356 - accuracy: 0.0177
Epoch 330/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2399 - accuracy: 0.0142
Epoch 331/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2306 - accuracy: 0.0177
Epoch 332/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2283 - accuracy: 0.0284
Epoch 333/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2392 - accuracy: 0.0248
Epoch 334/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2351 - accuracy: 0.0071
Epoc

57/57 [==============================] - 0s 1ms/step - loss: 4.2322 - accuracy: 0.0177
Epoch 488/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2427 - accuracy: 0.0177
Epoch 489/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2343 - accuracy: 0.0142
Epoch 490/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2233 - accuracy: 0.0142
Epoch 491/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2460 - accuracy: 0.0071
Epoch 492/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2334 - accuracy: 0.0284
Epoch 493/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2421 - accuracy: 0.0071
Epoch 494/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2447 - accuracy: 0.0142
Epoch 495/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2343 - accuracy: 0.0213
Epoch 496/500
57/57 [==============================] - 0s 1ms/step - loss: 4.2507 - accuracy: 0.0035
Epoc

In [13]:
# Saving the model
model.save('chatbot_model.h5', hist)
print("model created")

model created


# NEW CODE

In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import json
import pickle
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
#Loading intents.json
with open('intents.json') as intents:
  data = json.load(intents)

stemmer = LancasterStemmer()

# getting informations from intents.json--
words = []
labels = []
x_docs = []
y_docs = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [4]:
# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [5]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [6]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=800, batch_size=8, show_metric=True)

Training Step: 19999  | total loss: 0.07160 | time: 0.067s
| Adam | epoch: 800 | loss: 0.07160 - acc: 0.9713 -- iter: 192/197
Training Step: 20000  | total loss: 0.06457 | time: 0.070s
| Adam | epoch: 800 | loss: 0.06457 - acc: 0.9742 -- iter: 197/197
--


In [35]:
model.save('model.tflearn')

INFO:tensorflow:C:\Users\user\bangkit2023\ChoTracker_model\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Making Prediction

In [36]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

In [37]:
def chat():

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

In [ ]:
chat()



You: Hai
Bot:	Halo, ada yang bisa saya bantu?


You: Cholestrol itu apa
Bot:	Kolesterol HDL dianggap sebagai kolesterol 'baik' karena membantu menghilangkan kolesterol LDL dari aliran darah. Kolesterol HDL berperan sebagai pembersih, mengambil kelebihan kolesterol dan membawanya ke hati untuk diproses dan dikeluarkan.


You: apa itu kolesterol
Bot:	Kolesterol adalah zat lilin yang ditemukan dalam darah. Kolesterol berasal dari dua sumber, yaitu tubuh Anda dan makanan yang Anda konsumsi. Tubuh membutuhkan kolesterol untuk membentuk sel-sel yang sehat, tetapi kadar kolesterol yang tinggi dapat meningkatkan risiko penyakit jantung.


You: tips terhindar dari kolesterol
Bot:	Serat memiliki efek menguntungkan pada kolesterol dengan membantu mengikat kolesterol dalam saluran pencernaan. Makan makanan tinggi serat seperti buah-buahan, sayuran, dan biji-bijian dapat membantu menurunkan kadar kolesterol. Usahakan untuk mendapatkan setidaknya 25-30 gram serat per hari dalam diet Anda.


You: t